# Final Model Tuning and Accuracy

STAT 451: Machine Learning (Fall 2021)  
Instructor: Sebastian Raschka (sraschka@wisc.edu)   

---

This notebook is used to display tuning and model selection

In [177]:
%load_ext watermark
%watermark  -d -u -a 'William Ryan' -v -p numpy,scipy,matplotlib,sklearn,pandas,mlxtend

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: William Ryan

Last updated: 2021-12-06

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.28.0

numpy     : 1.20.3
scipy     : 1.7.1
matplotlib: 3.4.3
sklearn   : 1.0
pandas    : 1.3.3
mlxtend   : 0.19.0



In [148]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score

from mlxtend.evaluate import bootstrap_point632_score
from mlxtend.evaluate import mcnemar

In [2]:
data_path = '../data/Data/features_3_sec.csv'
df = pd.read_csv(data_path)
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [3]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df.label)
df['label'] = le.transform(df.label)
df['label']

0       0
1       0
2       0
3       0
4       0
       ..
9985    9
9986    9
9987    9
9988    9
9989    9
Name: label, Length: 9990, dtype: int64

In [17]:
y = df['label'].values
X = df.iloc[:, 1:-1].values

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y,
                     train_size=0.7,
                     stratify=y,
                     random_state=0)

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = \
    train_test_split(X_scaled, y,
                     train_size=0.7,
                     stratify=y,
                     random_state=0)

### Random Forest

We start with random forest. We removed many of the parameters that we searched in the interest of time. However, we performed a grid search cross validation for hyperparameter tuning

In [161]:
from sklearn.ensemble import RandomForestClassifier
import time

t1 = time.time()
rf = RandomForestClassifier()
#92 n: 85.919%
#412 n: 87.554%
#84: 86.812
#252: 86.883
#261: 86.919
#184: 86.991
#213: 87.312

params = {
    "n_estimators": [380],
    "random_state": [261],
    "max_depth": [20],
    "criterion": ["gini"],
    "oob_score": [True]
}

gs_rf = GridSearchCV(estimator=rf,
                  param_grid=params,
                  scoring='accuracy',
                  cv=5)

##k fold cross validation

gs_rf.fit(X_train, y_train)

print(str(time.time() - t1))
print('Best Accuracy: %.4f%%' % (gs_rf.best_score_*100))
print('Best Params: %s' % gs_rf.best_params_)
print(f"Test Accuracy: {gs_rf.score(X_test, y_test)*100:0.4f}%")

46.83710479736328
Best Accuracy: 85.1567%
Best Params: {'criterion': 'gini', 'max_depth': 20, 'n_estimators': 380, 'oob_score': True, 'random_state': 261}
Test Accuracy: 87.2873%


In [164]:
# F1-Score
f1_score(y_test, gs_rf.best_estimator_.predict(X_test), average='macro')

0.872145768758665

In [165]:
rf_ra = roc_auc_score(y_test, gs_rf.best_estimator_.predict_proba(X_test), multi_class='ovr')
rf_ra

0.9882112029792621

In [166]:
precision_score(y_test, gs_rf.best_estimator_.predict(X_test), average='macro')

0.8730109529336728

#### Random Forest Cross Validation

### KNN

Initial params

---

params = {
    
    "n_neighbors": range(1, 11),
    
    "weights": ["uniform", "distance"],
    
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
    
    "p": [1, 2]
    
}

In [172]:
from sklearn.neighbors import KNeighborsClassifier

t1 = time.time()

kn = KNeighborsClassifier()

params = {
    "n_neighbors": [1],
    "weights": ["uniform"],
    "algorithm": ["auto"],
    "p": [1]
    
}

gs_knn = GridSearchCV(estimator=kn,
                  param_grid=params,
                  scoring='accuracy',
                  cv=5)

##k fold cross validation

gs_knn.fit(X_train_scaled, y_train_scaled)

print(str(time.time() - t1))
print('Best Accuracy: %.3f%%' % (gs_knn.best_score_*100))
print('Best Params: %s' % gs_knn.best_params_)
print(f"Test Accuracy: {gs_knn.best_estimator_.score(X_test_scaled, y_test_scaled)*100:0.3f}%")

1.0184800624847412
Best Accuracy: 91.420%
Best Params: {'algorithm': 'auto', 'n_neighbors': 1, 'p': 1, 'weights': 'uniform'}
Test Accuracy: 92.759%


In [169]:
score = cross_val_score(gs_knn.best_estimator_, X_train_scaled, y_train_scaled, cv=5)
np.mean(score)

0.9142000059310706

In [170]:
accuracy_score(y_test_scaled, gs_knn.best_estimator_.predict(X_test_scaled))

0.9275942609275942

In [171]:
f1_score(y_test_scaled, gs_knn.best_estimator_.predict(X_test_scaled), average='macro')

0.9275324487942423

In [145]:
knn_ra = roc_auc_score(y_test_scaled, gs_knn.best_estimator_.predict_proba(X_test_scaled), multi_class='ovr')
knn_ra

0.9597600874471299

In [157]:
precision_score(y_test_scaled, gs_knn.best_estimator_.predict(X_test_scaled), average='macro')

0.9283392653340228

#### SVM

Initial Params listed below

---

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

In [160]:
from sklearn import svm

model = svm.SVC(decision_function_shape='ovo', random_state=0)
t1 = time.time()
model.fit(X_train_scaled, y_train_scaled)
print(str(time.time() - t1))
svm_score = model.score(X_test_scaled, y_test_scaled)
print(f"Train Accuracy: {model.score(X_train_scaled, y_train_scaled)*100:0.3f}%")
print(f"Test Accuracy: {svm_score*100:0.4f}%")

0.9676499366760254
Train Accuracy: 91.949%
Test Accuracy: 85.0517%


In [162]:
param_grid = {'C': [100],
              'gamma': [0.01],
              'kernel': ['rbf'],
              'probability': [True]}

gs_svm = GridSearchCV(estimator=model,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5)

gs_svm.fit(X_train_scaled, y_train_scaled)
print('Best Accuracy: %.4f%%' % (gs_svm.best_score_*100))
print('Best Params: %s' % gs_svm.best_params_)
print(f"Test Accuracy: {gs_svm.best_estimator_.score(X_test_scaled, y_test_scaled)*100:0.3f}%")

Best Accuracy: 88.1739%
Best Params: {'C': 100, 'gamma': 0.01, 'kernel': 'rbf', 'probability': True}
Test Accuracy: 90.724%


In [163]:
# f1-score
f1_score(y_test_scaled, gs_svm.best_estimator_.predict(X_test_scaled), average='macro')

0.9067111456351078

In [146]:
svm_ra = roc_auc_score(y_test_scaled, gs_svm.best_estimator_.predict_proba(X_test_scaled), multi_class='ovr')
svm_ra

0.9942948317259331

#### Gradient Boosting

Initial Params listed below

---

param_grid = {
    'classifier__learning_rate': (0.01, 0.1, 1, 10),
    'classifier__max_leaf_nodes': (3, 10, 30)
}

In [107]:
from sklearn.ensemble import HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier(random_state=123)
t1 = time.time()
clf = clf.fit(X_train, y_train)
print(str(time.time() - t1))
hist_gb_score = clf.score(X_test, y_test)
print(f"Train Accuracy: {clf.score(X_train, y_train)*100:0.3f}%")
print(f"Test Accuracy: {hist_gb_score*100:0.3f}%")

9.126726150512695
Train Accuracy: 99.914%
Test Accuracy: 89.623%


In [174]:
param_grid = {
    'learning_rate': [0.1],
    'max_leaf_nodes': [30]
}

gs_gb = GridSearchCV(estimator=clf,
                  param_grid=param_grid,
                  scoring='accuracy',
                  cv=5)

gs_gb.fit(X_train, y_train)
print('Best Accuracy: %.3f%%' % (gs_gb.best_score_*100))
print('Best Params: %s' % gs_gb.best_params_)
print(f"Test Accuracy: {gs_gb.best_estimator_.score(X_test, y_test)*100:0.3f}%")

Best Accuracy: 89.032%
Best Params: {'learning_rate': 0.1, 'max_leaf_nodes': 30}
Test Accuracy: 89.857%


In [168]:
f1_score(y_test, gs_gb.best_estimator_.predict(X_test), average='macro')

0.8983442385479691

In [147]:
gb_ra = roc_auc_score(y_test, gs_gb.best_estimator_.predict_proba(X_test), multi_class='ovr')
gb_ra

0.9935791461281662

### McNemar tests

##### Gradient Boosting and KNN

In [100]:
accuracy_score(y_test, gs_gb.best_estimator_.predict(X_test))
gb_pd = gs_gb.best_estimator_.predict(X_test)
knn_pd = gs_knn.best_estimator_.predict(X_test_scaled)
tb = np.array([[0, 0],
              [0, 0]])
for i in range(len(gb_pd)):
    t1 = gb_pd[i] == y_test[i]
    t2 = knn_pd[i] == y_test_scaled[i]
    if t1 and t2:
        tb[0][0] += 1
    elif t1:
        tb[1][0] += 1
    elif t2:
        tb[0][1] += 1
    else:
        tb[1][1] += 1
tb

array([[2563,  217],
       [ 130,   87]])

In [101]:
from mlxtend.evaluate import mcnemar

chi2, p = mcnemar(ary=tb, corrected=True)
print('chi-squared:', chi2)
print('p-value:', p)

chi-squared: 21.314121037463977
p-value: 3.898491498111577e-06


##### KNN and SVM

In [102]:
svm_p = gs_svm.best_estimator_.predict(X_test_scaled)
knn_p = gs_knn.best_estimator_.predict(X_test_scaled)
tb = mcnemar_table(y_target=y_test_scaled, 
                   y_model1=svm_p, 
                   y_model2=knn_p)
tb

array([[2602,  117],
       [ 178,  100]])

In [103]:
chi2, p = mcnemar(ary=tb, corrected=True)
print('chi-squared:', chi2)
print('p-value:', p)

chi-squared: 12.203389830508474
p-value: 0.0004770275703526321


##### Gradient Boosting and SVM

In [104]:
gb_pd = gs_gb.best_estimator_.predict(X_test)
svm_pd = gs_svm.best_estimator_.predict(X_test_scaled)
tb = np.array([[0, 0],
              [0, 0]])
for i in range(len(gb_pd)):
    t1 = gb_pd[i] == y_test[i]
    t2 = svm_pd[i] == y_test_scaled[i]
    if t1 and t2:
        tb[0][0] += 1
    elif t1:
        tb[1][0] += 1
    elif t2:
        tb[0][1] += 1
    else:
        tb[1][1] += 1
tb

array([[2557,  162],
       [ 136,  142]])

In [105]:
from mlxtend.evaluate import mcnemar

chi2, p = mcnemar(ary=tb, corrected=True)
print('chi-squared:', chi2)
print('p-value:', p)

chi-squared: 2.097315436241611
p-value: 0.14755801673609054
